<a href="https://colab.research.google.com/github/Ayesha-Imr/Gen-AI-Projects/blob/main/Youtube_Transcript_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet youtube-transcript-api

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

YouTubeTranscriptApi.get_transcript('iCzUv6-9zrI')

[{'text': '(upbeat music)', 'start': 0.716, 'duration': 2.583},
 {'text': '- All right, our last talk for the day.',
  'start': 10.17,
  'duration': 3.27},
 {'text': "My name's Ben Schmerler.", 'start': 13.44, 'duration': 0.96},
 {'text': 'I just did the talk a little bit ago',
  'start': 14.4,
  'duration': 2.61},
 {'text': 'about toilet hacking.', 'start': 17.01, 'duration': 1.353},
 {'text': 'I work with ISC.', 'start': 19.404, 'duration': 0.833},
 {'text': "You wanna talk to me more, I'm over there.",
  'start': 20.237,
  'duration': 1.243},
 {'text': 'But today, or for this last session,',
  'start': 21.48,
  'duration': 2.04},
 {'text': "I'm introducing Lenin Alevski at Google",
  'start': 23.52,
  'duration': 2.25},
 {'text': 'and Max vonBlankenburg.', 'start': 25.77, 'duration': 1.38},
 {'text': 'We are gonna talk about\nsome security stuff,',
  'start': 27.15,
  'duration': 1.77},
 {'text': 'so please welcome to the stage.',
  'start': 28.92,
  'duration': 2.302},
 {'text': '(

In [ ]:
!pip install --quiet replicate langchain langchain-community weaviate-client llama_index ftfy regex tqdm git+https://github.com/openai/CLIP.git torch torchvision matplotlib scikit-image PyMuPDF

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 667.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.7/325.7 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.9 MB

In [ ]:
import os
import torch
import clip
from youtube_transcript_api import YouTubeTranscriptApi
from datetime import datetime
from weaviate.util import generate_uuid5
import weaviate

In [ ]:
import os

from google.colab import userdata
WCS_URL =  userdata.get('WCS_URL')
WCS_API_KEY =  userdata.get('WCS_API_KEY')

os.environ["WCS_URL"] = WCS_URL
os.environ["WCS_API_KEY"] = WCS_API_KEY

In [ ]:
import weaviate
from weaviate.auth import AuthApiKey

# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.getenv("WCS_URL"),
    auth_credentials=AuthApiKey(os.getenv("WCS_API_KEY")),
)

print(client.is_ready())

True


In [ ]:
# Function to get current time in RFC3339 format
def get_rfc3339_timestamp():
    return datetime.now().isoformat(timespec='seconds') + 'Z'

# Extract YouTube transcript
video_id = 'iCzUv6-9zrI'
transcript = YouTubeTranscriptApi.get_transcript(video_id)
transcript_texts = [entry['text'] for entry in transcript]

In [ ]:
# Setup CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# Tokenize and encode transcript text
text_tokens = clip.tokenize(transcript_texts).to(device)
with torch.no_grad():
    text_embeddings = model.encode_text(text_tokens)

# Convert embeddings to numpy for ingestion
text_embeddings_np = text_embeddings.cpu().numpy()


100%|███████████████████████████████████████| 338M/338M [00:12<00:00, 28.9MiB/s]


In [ ]:
# Upload transcript embeddings with metadata in batch
text_collection = client.collections.get("RSATextCollection")
with text_collection.batch.dynamic() as batch:
  for idx, (text, embedding) in enumerate(zip(transcript_texts, text_embeddings_np)):
      if idx % 10 == 0:
          print(idx, text, embedding.shape)
      obj_uuid = generate_uuid5(f"{text}_{idx}")  # Ensure unique UUID by including the index
      batch.add_object(
          properties={
              "text": text,
              "source": f"https://www.youtube.com/watch?v={video_id}",
              "timestamp": get_rfc3339_timestamp()
          },
          uuid=obj_uuid,
          vector=embedding.tolist()
      )

print("YouTube transcript embeddings uploaded successfully")

0 (upbeat music) (512,)
10 We are gonna talk about
some security stuff, (512,)
20 the opinions expressed
here are not the opinions (512,)
30 So probably if you are here, (512,)
40 where we have a special component, (512,)
50 A container is a special
format for your application (512,)
60 So yeah, what containers really are? (512,)
70 and finally, capabilities which control (512,)
80 It's an orchestration tool, a framework (512,)
90 refer to the way Kubernetes allows users (512,)
100 of a Kubernetes application in general. (512,)
110 a Kubernetes application
looks under the hood. (512,)
120 These include the kube-apiserver (512,)
130 according to your deployment. (512,)
140 and controls your node. (512,)
150 they control all the
pods running on said node (512,)
160 the Kubernetes most
common attack techniques. (512,)
170 to the machine becomes
initial access to the cluster, (512,)
180 that they were not relevant anymore. (512,)
190 For example, when it comes
to using cloud credentials, (

In [ ]:
# Function to embed text query
def embed_text_query(query, model, device):
    query_token = clip.tokenize([query]).to(device)
    with torch.no_grad():
        query_embedding = model.encode_text(query_token)
    return query_embedding.cpu().numpy()

In [ ]:
import weaviate.classes as wvc

def search_weaviate(query, model, preprocess, client, device, collection_name):
    query_embedding = embed_text_query(query, model, device)

    near_vector =  query_embedding.flatten().tolist()

    query_collection = client.collections.get(collection_name)
    response = query_collection.query.near_vector(
        near_vector=near_vector,
        limit=15,
        return_metadata=wvc.query.MetadataQuery(certainty=True)
    )
    return response


In [ ]:
import json

# Example usage
collection_name = "RSATextCollection"  # Change to "RSAImageCollection" if searching images
query = "what does Max vonBlankenburg say about kubernetes"  # Change to the path of an image file if searching images

response = search_weaviate(query, model, preprocess, client, device, collection_name)
print(response)


QueryReturn(objects=[Object(uuid=_WeaviateUUIDInt('04902669-f432-52f4-9968-4d7226f284a2'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=0.962879478931427, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'text': 'What Is Kubernetes Anyways?', 'timestamp': datetime.datetime(2024, 7, 1, 9, 1, 46, tzinfo=datetime.timezone.utc), 'source': 'rsa-doc.pdf'}, references=None, vector={}, collection='RSATextCollection'), Object(uuid=_WeaviateUUIDInt('86725242-be37-5a96-9511-f50e9582f8a7'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=0.9574028849601746, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'text': 'what the hell is Kubernetes,\nthe Kubernetes threat model.', 'source': 'https://www.youtube.com/watch?v=iCzUv6-9zrI', 'timestamp': datetime.datetime(2024, 7, 6, 16, 36, 45, tzinfo=datetime.timezone.utc)}, references=None, vector=

In [ ]:
def print_formatted_response(response):
    # Check if response is valid and has objects
    if not response or not hasattr(response, 'objects') or not response.objects:
        print("No valid response data.")
        return

    for obj in response.objects:
        properties = obj.properties
        print("Text:", properties.get('text', 'N/A'))
        print("Source:", properties.get('source', 'N/A'))
        print("Timestamp:", properties.get('timestamp', 'N/A'))
        print("Certainty:", getattr(obj.metadata, 'certainty', 'N/A'))
        print("-" * 50)


print_formatted_response(response)

Text: What Is Kubernetes Anyways?
Source: rsa-doc.pdf
Timestamp: 2024-07-01 09:01:46+00:00
Certainty: 0.962879478931427
--------------------------------------------------
Text: what the hell is Kubernetes,
the Kubernetes threat model.
Source: https://www.youtube.com/watch?v=iCzUv6-9zrI
Timestamp: 2024-07-06 16:36:45+00:00
Certainty: 0.9574028849601746
--------------------------------------------------
Text: You can probably think of Kubernetes.
Source: https://www.youtube.com/watch?v=iCzUv6-9zrI
Timestamp: 2024-07-06 16:36:45+00:00
Certainty: 0.9573091268539429
--------------------------------------------------
Text: Now let's explain what Kubernetes is.
Source: https://www.youtube.com/watch?v=iCzUv6-9zrI
Timestamp: 2024-07-06 16:36:45+00:00
Certainty: 0.9551944732666016
--------------------------------------------------
Text: refer to the way Kubernetes allows users
Source: https://www.youtube.com/watch?v=iCzUv6-9zrI
Timestamp: 2024-07-06 16:36:45+00:00
Certainty: 0.9537145495414734
-